In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

2022-12-07 10:21:24.155932: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from IPython.display import HTML

In [4]:
def createModel(input_shape = (203, 23*7)):
    
    X = tfl.Input(input_shape)  # define the input to the model
    lstm = tfl.LSTM(100, activation='tanh', recurrent_activation='tanh', return_sequences=True)(X)
    #drop = tfl.Dropout(0.2)(lstm)
    d2 = tfl.Dense(50,activation=None)(lstm)
    d3 = tfl.Dense(3,activation=None)(d2)
    permute = tfl.Permute((2,1))(d3)    # change input from (None, 203, 3) to (None, 3, 203)
    
    # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
    # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
    probs = tfl.Cropping1D(cropping=(0,1))(permute) # shape (None, 2, 203)
    probs = tfl.Softmax(axis=1)(probs)
    
    time = tfl.Cropping1D(cropping=(2,0))(permute) # shape (None, 1, 203)
    
    # concatenate the probabilities and predicted_time_to_sack back into one layer
    out = tfl.Concatenate(axis=1)([probs, time]) # shape (None, 3, 203)
    out = tfl.Permute((2,1))(out) # shape (None, 203, 3)
    
    model = Model(inputs=X, outputs=out)        # create model
    
    return model

model = createModel()

2022-12-07 10:21:39.194630: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model_string = f"./rnn_model_unnorm/weights/weights_epochs2"

model.load_weights(model_string)

In [6]:
MAX_PLAY_LENGTH = 203

x_val = np.load("./seq_unnorm_data/x_val.npy")
y_val = np.load("./seq_unnorm_data/y_val.npy")
#train_mu = np.load("./seq_data/train_mu.npy")
#train_std = np.load("./seq_data/train_std.npy")


In [58]:
x_train = np.load("./seq_unnorm_data/x_train.npy")
y_train = np.load("./seq_unnorm_data/y_train.npy")

In [59]:
x_train_input = x_train.reshape(-1, MAX_PLAY_LENGTH, 23, 11)[:,:,:,4:].reshape(-1,MAX_PLAY_LENGTH,23*7)

In [7]:
x_val_input = x_val.reshape(-1, MAX_PLAY_LENGTH, 23, 11)[:,:,:,4:].reshape(-1,MAX_PLAY_LENGTH,23*7)

In [60]:
train_pred = model.predict(x_train_input)

188/188 [==============================] - 17s 89ms/step


In [61]:
train_pred[train_pred[:,:,1] > 0.5]

array([], shape=(0, 3), dtype=float32)

In [8]:
x_val[0][-1]

array([ 2.0211017e+09,  2.3270000e+03,  2.8000000e+01,  4.3290000e+04,
        1.0000000e+00,  8.5840000e+01,  1.8900000e+00,  6.8000000e-01,
        1.5400000e+00,  1.5770000e+01,  2.5118000e+02,  2.0211017e+09,
        2.3270000e+03,  2.8000000e+01,  4.3299000e+04,  2.0000000e+00,
        6.8830000e+01,  1.7170000e+01,  5.2500000e+00,  4.6100000e+00,
        2.7461000e+02,  3.0631000e+02,  2.0211017e+09,  2.3270000e+03,
        2.8000000e+01,  4.3350000e+04,  2.0000000e+00,  6.8740000e+01,
        4.3600000e+00,  2.5200000e+00,  7.4000000e-01,  2.1797000e+02,
        2.7068000e+02,  2.0211017e+09,  2.3270000e+03,  2.8000000e+01,
        4.3453000e+04,  1.0000000e+00,  8.4120000e+01,  4.2900000e+00,
        2.7800000e+00,  2.2400000e+00,  3.4785000e+02,  1.7948000e+02,
        2.0211017e+09,  2.3270000e+03,  2.8000000e+01,  4.3455000e+04,
        2.0000000e+00,  8.3440000e+01,  3.9500000e+00,  3.3200000e+00,
        1.4200000e+00,  1.4426000e+02,  2.0401000e+02,  2.0211017e+09,
      

In [9]:
preds = model.predict(x_val_input)

41/41 [==============================] - 4s 69ms/step


In [57]:
preds[preds[:,:,1] > 0.5]

array([], shape=(0, 3), dtype=float32)

In [56]:
i = 8
preds[i][-20:]

array([[ 0.9053695 ,  0.09463044, -0.5875259 ],
       [ 0.9314897 ,  0.0685103 , -0.64179426],
       [ 0.90527105,  0.09472892, -0.6313407 ],
       [ 0.92574286,  0.0742571 , -0.48950994],
       [ 0.9083827 ,  0.09161732, -0.5801849 ],
       [ 0.9295971 ,  0.07040295, -0.55132955],
       [ 0.90419775,  0.09580225, -0.5707871 ],
       [ 0.9254991 ,  0.07450088, -0.5152909 ],
       [ 0.8812135 ,  0.11878654, -0.63681036],
       [ 0.8746549 ,  0.12534508, -0.44868794],
       [ 0.86104643,  0.13895358, -0.7284291 ],
       [ 0.8763989 ,  0.12360106, -0.4705741 ],
       [ 0.8666027 ,  0.13339724, -0.7265541 ],
       [ 0.884746  ,  0.11525399, -0.5256022 ],
       [ 0.90284485,  0.09715517, -0.6951967 ],
       [ 0.8867313 ,  0.11326862, -0.5250653 ],
       [ 0.9035177 ,  0.09648226, -0.693719  ],
       [ 0.8937884 ,  0.10621166, -0.5156545 ],
       [ 0.91133064,  0.08866932, -0.6803594 ],
       [ 0.90670645,  0.09329352, -0.52802557]], dtype=float32)

In [10]:
print(x_val.shape)
print(x_val_input.shape)
print(preds.shape)

(1300, 203, 253)
(1300, 203, 161)
(1300, 203, 3)


In [11]:
preds[0:23][-1]

array([[ 5.09963810e-01,  4.90036249e-01, -5.88438066e-04],
       [ 5.10057688e-01,  4.89942342e-01, -5.17557492e-04],
       [ 5.10129213e-01,  4.89870787e-01, -4.63713077e-04],
       [ 5.10183632e-01,  4.89816338e-01, -4.22814046e-04],
       [ 5.10225058e-01,  4.89774942e-01, -3.91745649e-04],
       [ 5.10256529e-01,  4.89743471e-01, -3.68141831e-04],
       [ 5.10280430e-01,  4.89719540e-01, -3.50207032e-04],
       [ 5.10298669e-01,  4.89701390e-01, -3.36578756e-04],
       [ 5.10312438e-01,  4.89687562e-01, -3.26222391e-04],
       [ 5.10322928e-01,  4.89677101e-01, -3.18352715e-04],
       [ 5.10330915e-01,  4.89669114e-01, -3.12372809e-04],
       [ 5.10336936e-01,  4.89663064e-01, -3.07829410e-04],
       [ 5.10341525e-01,  4.89658475e-01, -3.04377929e-04],
       [ 5.10344982e-01,  4.89654958e-01, -3.01756314e-04],
       [ 5.10347664e-01,  4.89652336e-01, -2.99765437e-04],
       [ 5.10349691e-01,  4.89650339e-01, -2.98253901e-04],
       [ 5.10351181e-01,  4.89648819e-01

In [12]:
# unadjust coordinates
# unadj_x_val = x_val
# unadj_x_val[:,:,5:] = (x_val[:,:,5:]*train_std[5:])+train_mu[5:]
# unadj_x_val[:,:,6] += (53.3/2)

        

In [13]:
four_dim = x_val.reshape(x_val.shape[0], MAX_PLAY_LENGTH, 23, -1)

In [14]:
four_dim[:,:,:,6] += (53.3/2)

In [15]:
four_dim[3][-1][0:3]

array([[2.0211017e+09, 2.5860000e+03, 3.4000000e+01, 3.8553000e+04,
        2.0000000e+00, 5.8410000e+01, 2.3530000e+01, 1.9700000e+00,
        1.5000000e-01, 2.7986000e+02, 2.6630000e+02],
       [2.0211017e+09, 2.5860000e+03, 3.4000000e+01, 4.0171000e+04,
        1.0000000e+00, 5.8180000e+01, 2.2520000e+01, 2.4600000e+00,
        7.6000000e-01, 1.4206000e+02, 2.3960000e+02],
       [2.0211017e+09, 2.5860000e+03, 3.4000000e+01, 4.1619000e+04,
        2.0000000e+00, 6.0080000e+01, 2.3290000e+01, 1.3700000e+00,
        5.4000000e-01, 3.0715000e+02, 1.8863000e+02]])

In [16]:
four_dim.shape

(1300, 203, 23, 11)

In [17]:
test_preds = np.tile(preds, (1,1,23)).reshape(1300, 203, 23, 3)

In [18]:
test_preds.shape

(1300, 203, 23, 3)

In [19]:
test_preds[0][0]

array([[ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.00058844],
       [ 0.5099638 ,  0.49003625, -0.000

In [20]:
# Doesn't wokr properly
# preds_repeat = np.repeat(preds, 23, axis=2).reshape(-1, MAX_PLAY_LENGTH, 23, 3)

# preds_repeat.shape

In [21]:
# preds_repeat[0][0]

In [22]:
val_with_preds = np.concatenate([four_dim, test_preds], axis=3)

val_with_preds.shape

(1300, 203, 23, 14)

In [23]:
val_with_preds[0][-1]

array([[ 2.02110170e+09,  2.32700000e+03,  2.80000000e+01,
         4.32900000e+04,  1.00000000e+00,  8.58400000e+01,
         2.85400000e+01,  6.80000000e-01,  1.54000000e+00,
         1.57700000e+01,  2.51180000e+02,  9.00751233e-01,
         9.92487743e-02, -8.25156748e-01],
       [ 2.02110170e+09,  2.32700000e+03,  2.80000000e+01,
         4.32990000e+04,  2.00000000e+00,  6.88300000e+01,
         4.38200000e+01,  5.25000000e+00,  4.61000000e+00,
         2.74610000e+02,  3.06310000e+02,  9.00751233e-01,
         9.92487743e-02, -8.25156748e-01],
       [ 2.02110170e+09,  2.32700000e+03,  2.80000000e+01,
         4.33500000e+04,  2.00000000e+00,  6.87400000e+01,
         3.10100000e+01,  2.52000000e+00,  7.40000000e-01,
         2.17970000e+02,  2.70680000e+02,  9.00751233e-01,
         9.92487743e-02, -8.25156748e-01],
       [ 2.02110170e+09,  2.32700000e+03,  2.80000000e+01,
         4.34530000e+04,  1.00000000e+00,  8.41200000e+01,
         3.09400000e+01,  2.78000000e+00,  2.

In [24]:
# import itertools
# perms = list(itertools.permutations([0,1,2,3]))

# perms[0]

In [25]:
# for perm in perms:
#     test = val_with_preds.transpose(perm).reshape(-1,val_with_preds.shape[-1])
#     print(f"Perm {perm} : \n \t test = {test[-1]}")

In [26]:
col_names = ['gameId', 'playId', 'frameId', 'nflId', 'team_indicator', 'adj_x', 'adj_y', 's', 'a', 'adj_o', 'adj_dir', 'no_sack_prob', 'sack_prob', 'pred_time']

val_with_preds_df = pd.DataFrame(val_with_preds.reshape(-1, val_with_preds.shape[-1]), columns=col_names)
val_with_preds_df.gameId.astype('int64', copy=False)


0                   0
1                   0
2                   0
3                   0
4                   0
              ...    
6069695    2021102408
6069696    2021102408
6069697    2021102408
6069698    2021102408
6069699    2021102408
Name: gameId, Length: 6069700, dtype: int64

In [27]:
val_with_preds_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time
0,0.0,0.0,0.0,0.0,0.0,0.0,26.65,0.0,0.0,0.0,0.0,0.509964,0.490036,-0.000588
1,0.0,0.0,0.0,0.0,0.0,0.0,26.65,0.0,0.0,0.0,0.0,0.509964,0.490036,-0.000588
2,0.0,0.0,0.0,0.0,0.0,0.0,26.65,0.0,0.0,0.0,0.0,0.509964,0.490036,-0.000588
3,0.0,0.0,0.0,0.0,0.0,0.0,26.65,0.0,0.0,0.0,0.0,0.509964,0.490036,-0.000588
4,0.0,0.0,0.0,0.0,0.0,0.0,26.65,0.0,0.0,0.0,0.0,0.509964,0.490036,-0.000588


In [28]:
y_val.shape

(1300, 203, 3)

In [29]:
val_with_preds_df['true_sack'] = np.tile(y_val, (1,1,23)).reshape(-1,3)[:,1]

In [30]:
val_with_preds_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
0,0.0,0.0,0.0,0.0,0.0,0.0,26.65,0.0,0.0,0.0,0.0,0.509964,0.490036,-0.000588,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,26.65,0.0,0.0,0.0,0.0,0.509964,0.490036,-0.000588,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,26.65,0.0,0.0,0.0,0.0,0.509964,0.490036,-0.000588,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,26.65,0.0,0.0,0.0,0.0,0.509964,0.490036,-0.000588,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,26.65,0.0,0.0,0.0,0.0,0.509964,0.490036,-0.000588,0.0


In [31]:
val_with_preds_no_padding_df = val_with_preds_df[val_with_preds_df.gameId != 0]
val_with_preds_no_padding_df.gameId = val_with_preds_no_padding_df.gameId.astype('int64')

/var/folders/59/2qy07m9n56j1hjrbkcj93rhc0000gn/T/ipykernel_9615/3598514011.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_with_preds_no_padding_df.gameId = val_with_preds_no_padding_df.gameId.astype('int64')


In [32]:
val_with_preds_no_padding_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
4140,2021101704,2327.0,6.0,43290.0,1.0,83.53,29.44,0.01,0.01,357.97,247.63,0.887123,0.112877,-0.472715,0.0
4141,2021101704,2327.0,6.0,43299.0,2.0,76.40,44.71,1.93,1.42,235.91,21.00,0.887123,0.112877,-0.472715,0.0
4142,2021101704,2327.0,6.0,43350.0,2.0,67.91,35.25,0.32,0.69,170.93,150.43,0.887123,0.112877,-0.472715,0.0
4143,2021101704,2327.0,6.0,43453.0,1.0,80.54,30.92,0.09,0.54,26.79,291.61,0.887123,0.112877,-0.472715,0.0
4144,2021101704,2327.0,6.0,43455.0,2.0,78.70,30.61,0.04,0.45,206.57,204.82,0.887123,0.112877,-0.472715,0.0


In [33]:
true_sack_df = val_with_preds_df.query("true_sack == 1")
true_sack_df.gameId = true_sack_df.gameId.astype('int64')

true_sack_df.head()

/var/folders/59/2qy07m9n56j1hjrbkcj93rhc0000gn/T/ipykernel_9615/3767260491.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_sack_df.gameId = true_sack_df.gameId.astype('int64')


,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
54855,2021101704,3012.0,6.0,38544.0,1.0,13.21,27.53,0.37,1.87,151.07,190.98,0.863566,0.136434,-0.655674,1.0
54856,2021101704,3012.0,6.0,38553.0,2.0,15.27,32.76,0.31,1.42,345.71,104.15,0.863566,0.136434,-0.655674,1.0
54857,2021101704,3012.0,6.0,40171.0,1.0,13.36,32.21,0.24,1.33,150.47,195.50,0.863566,0.136434,-0.655674,1.0
54858,2021101704,3012.0,6.0,41619.0,2.0,14.60,29.53,0.19,0.72,335.83,157.79,0.863566,0.136434,-0.655674,1.0
54859,2021101704,3012.0,6.0,42500.0,2.0,15.76,24.78,0.00,0.00,358.96,308.54,0.863566,0.136434,-0.655674,1.0


In [41]:
true_sack_df.drop_duplicates(['gameId', 'playId'])

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
54855,2021101704,3012.0,6.0,38544.0,1.0,13.21,27.53,0.37,1.87,151.07,190.98,0.863566,0.136434,-0.655674,1.0
190394,2021101705,206.0,6.0,38622.0,1.0,72.54,19.49,0.00,0.00,30.20,158.84,0.858013,0.141987,-0.453743,1.0
218684,2021101705,585.0,6.0,39971.0,1.0,46.43,10.10,1.59,0.28,156.49,338.23,0.858986,0.141014,-0.485437,1.0
325956,2021101705,2035.0,6.0,39971.0,1.0,62.91,9.78,0.17,0.84,156.73,110.77,0.854895,0.145105,-0.415296,1.0
334604,2021101705,2194.0,6.0,38622.0,1.0,76.70,34.33,0.00,0.00,353.46,290.30,0.849718,0.150282,-0.404730,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970731,2021102407,3881.0,6.0,37266.0,2.0,71.50,29.39,0.40,0.96,313.34,188.42,0.903473,0.096527,-0.556636,1.0
5984899,2021102408,131.0,6.0,37077.0,1.0,71.18,45.30,0.00,0.00,8.63,11.26,0.847386,0.152614,-0.397751,1.0
5998699,2021102408,273.0,6.0,33566.0,2.0,82.22,18.73,0.00,0.14,6.54,2.99,0.740649,0.259351,-0.598773,1.0
6002563,2021102408,353.0,6.0,37077.0,1.0,78.86,39.31,0.08,0.77,350.92,312.09,0.891230,0.108770,-0.446000,1.0


In [42]:
test_game_id = 2021101705
test_play_id = 206.0

In [43]:

play_df = val_with_preds_no_padding_df.query("gameId == @test_game_id and playId == @test_play_id")
print(f"play_df shape = {play_df.shape}")

play_df.head()

play_df shape = (1035, 15)


,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
190394,2021101705,206.0,6.0,38622.0,1.0,72.54,19.49,0.00,0.00,30.20,158.84,0.858013,0.141987,-0.453743,1.0
190395,2021101705,206.0,6.0,39947.0,1.0,72.19,26.23,0.09,0.71,27.68,105.23,0.858013,0.141987,-0.453743,1.0
190396,2021101705,206.0,6.0,41300.0,2.0,67.02,26.76,0.00,0.00,285.87,293.55,0.858013,0.141987,-0.453743,1.0
190397,2021101705,206.0,6.0,41308.0,2.0,70.25,36.13,0.02,0.02,204.25,134.83,0.858013,0.141987,-0.453743,1.0
190398,2021101705,206.0,6.0,41483.0,2.0,66.17,39.24,0.23,0.22,233.61,16.90,0.858013,0.141987,-0.453743,1.0


In [44]:
play_df.tail()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
191424,2021101705,206.0,50.0,52449.0,1.0,70.36,14.77,7.56,1.96,199.97,283.86,0.879487,0.120513,-0.729179,1.0
191425,2021101705,206.0,50.0,52498.0,2.0,79.18,21.88,5.79,4.15,285.03,292.96,0.879487,0.120513,-0.729179,1.0
191426,2021101705,206.0,50.0,53556.0,1.0,57.45,39.98,7.21,2.99,95.67,68.05,0.879487,0.120513,-0.729179,1.0
191427,2021101705,206.0,50.0,53624.0,2.0,71.49,26.91,1.53,0.24,180.14,277.28,0.879487,0.120513,-0.729179,1.0
191428,2021101705,206.0,50.0,0.0,3.0,78.97,21.27,5.08,2.82,0.00,0.00,0.879487,0.120513,-0.729179,1.0


In [45]:
animated_play = AnimateFeature(play_df)
HTML(animated_play.ani.to_jshtml())

In [37]:
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.patches as patches

In [38]:
class AnimateFeature():
    
    # initialize variables we need. 
    # Example: Need to initialize the figure that the animation command will return
    def __init__(self, play_df, displayNumbers=False) -> None:
        
        self.MAX_FIELD_PLAYERS = 22
        self.start_x = 0
        self.stop_x = 120
        self.start_y = 53.3
        self.play_df = play_df
        self.frames_list = play_df.frameId.unique()
        self.games_df = pd.read_csv("~/Documents/Python/nfl-big-data-bowl-2023/data/games.csv")
        self.predictions = True
        self.displayNumbers = displayNumbers
        self.FIG_HEIGHT = 4
        self.FIG_WIDTH = 8
        
        fig, ax = plt.subplots(1, figsize=(self.FIG_WIDTH, self.FIG_HEIGHT))
        
        self.fig = fig
        self.field_ax = ax
        
        # create new axis for home, away, jersey
        self.ax_home = self.field_ax.twinx()
        self.ax_away = self.field_ax.twinx()
        self.ax_jersey = self.field_ax.twinx()
        self.ax_pred = self.field_ax.twinx()
        
        self.ani = animation.FuncAnimation(self.fig, self.update, frames=len(self.frames_list),
                                          init_func=self.setup_plot, blit=False)
        
        plt.close()
        
    # initialization function for animation call
    def setup_plot(self):

        endzones = True
        linenumbers = True
        
        # set axis limits
        self.set_axis_plots(self.field_ax, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_home, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_away, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_jersey, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_pred, self.stop_x, self.start_y)

        # set up colors and patches for field
        self.set_up_field(endzones, linenumbers)
        
        # create scatterplots on axis for data
        self.scat_field = self.field_ax.scatter([], [], s=50, color='orange')
        self.scat_home = self.ax_home.scatter([], [], s=50, color='blue')
        self.scat_away = self.ax_away.scatter([], [], s=50, color='red')
        
        # create box for prediction
        self.scat_pred = self.ax_pred.text(0, 0, '', fontsize = self.FIG_WIDTH, 
                                           bbox=dict(boxstyle="square", facecolor="white"),
                                           horizontalalignment = 'left', verticalalignment = 'top', c = 'black')
        
        # add direction stats and jersey numbers/names
        self._scat_jersey_list = []
        self._scat_number_list = []
        self._scat_name_list = []
        self._a_dir_list = []
        self._a_or_list = []
        for _ in range(self.MAX_FIELD_PLAYERS):
            self._scat_jersey_list.append(self.ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'white'))
            self._scat_number_list.append(self.ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black'))
            self._scat_name_list.append(self.ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black'))
            
            self._a_dir_list.append(self.field_ax.add_patch(patches.Arrow(0, 0, 0, 0, color = 'k')))
            self._a_or_list.append(self.field_ax.add_patch(patches.Arrow(0, 0, 0, 0, color = 'k')))
        
        # return all axis plots that you want to update
        return (self.scat_field, self.scat_home, self.scat_away, self.ax_pred, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)
    
    def update(self, i):
        frame = self.frames_list[i]
        time_df = self.play_df.query("frameId == @frame")
        
        #time_df['team_indicator'] = self.add_team_indicator(time_df)
        #print(time_df)
        
        label_list = time_df.team_indicator.unique()
        #print(label_list)
        label1= label_list[0]
        label2 = label_list[1]
        label3 = label_list[2]
        
        # update each team/football x,y coordinates
        for label in label_list:
            label_data = time_df[time_df.team_indicator == label]
            
            if label == label1:
                self.scat_field.set_offsets(label_data[['adj_x','adj_y']].to_numpy())
            elif label == label2:
                self.scat_home.set_offsets(label_data[['adj_x','adj_y']].to_numpy())
            elif label == label3:
                self.scat_away.set_offsets(label_data[['adj_x','adj_y']].to_numpy())
                
        # add prediction
        if self.predictions != None:
            sack_prob = np.round(time_df.sack_prob.values[0], 3)
            time = np.round(time_df.pred_time.values[0], 3)
            true_sack = time_df.true_sack.values[0]
            set_str = f"Sack = {true_sack}, P(sack) = {sack_prob}, time = {time}"
            self.scat_pred.set_text(set_str)
        
        #add direction and jersey info
        jersey_df = time_df[time_df.nflId != 0].reset_index()
        
        for (index, row) in jersey_df.iterrows():
            #self._scat_jersey_list[index].set_position((row.x, row.y))
            #self._scat_jersey_list[index].set_text(row.position)
            
            if self.displayNumbers:
                self._scat_number_list[index].set_position((row.adj_x, row.adj_y))
                self._scat_number_list[index].set_text(int(row.nflId))
            #self._scat_name_list[index].set_position((row.x, row.y-1.9))
            #self._scat_name_list[index].set_text(row.displayName.split()[-1])
            
            player_orientation_rad = self.deg_to_rad(self.convert_orientation(row.adj_o))
            player_direction_rad = self.deg_to_rad(self.convert_orientation(row.adj_dir))
            player_speed = row.s
            
            player_vel = np.array([np.real(self.polar_to_z(player_speed, player_direction_rad)), np.imag(self.polar_to_z(player_speed, player_direction_rad))])
            player_orient = np.array([np.real(self.polar_to_z(2, player_orientation_rad)), np.imag(self.polar_to_z(2, player_orientation_rad))])
            
            self._a_dir_list[index].remove()
            self._a_dir_list[index] = self.field_ax.add_patch(patches.Arrow(row.adj_x, row.adj_y, player_vel[0], player_vel[1], color = 'k'))
            
            self._a_or_list[index].remove()
            self._a_or_list[index] = self.field_ax.add_patch(patches.Arrow(row.adj_x, row.adj_y, player_orient[0], player_orient[1], color = 'grey', width = 2))
                

        return (self.scat_field, self.scat_home, self.scat_away, self.ax_pred, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)
    
    def set_up_field(self, endzones=True, linenumbers=True) -> None:
        yard_numbers_size = self.fig.get_size_inches()[0]*1.5

        # color field 
        rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                                    edgecolor='r', facecolor='darkgreen', zorder=0)
        self.field_ax.add_patch(rect)

        # plot
        self.field_ax.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
                    80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
                    [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
                    53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
                    color='white')

        # Endzones
        if endzones:
            ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                    linewidth=0.1,
                                    edgecolor='r',
                                    facecolor='blue',
                                    alpha=0.2,
                                    zorder=0)
            ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                    linewidth=0.1,
                                    edgecolor='r',
                                    facecolor='blue',
                                    alpha=0.2,
                                    zorder=0)
            self.field_ax.add_patch(ez1)
            self.field_ax.add_patch(ez2)
            
        if endzones:
            hash_range = range(11, 110)
        else:
            hash_range = range(1, 120)

        # add hashes
        for x in hash_range:
            self.field_ax.plot([x, x], [0.4, 0.7], color='white')
            self.field_ax.plot([x, x], [53.0, 52.5], color='white')
            self.field_ax.plot([x, x], [22.91, 23.57], color='white')
            self.field_ax.plot([x, x], [29.73, 30.39], color='white')
            
        # add linenumbers
        if linenumbers:
                for x in range(20, 110, 10):
                    numb = x
                    if x > 50:
                        numb = 120 - x
                    self.field_ax.text(x, 5, str(numb - 10),
                            horizontalalignment='center',
                            fontsize=yard_numbers_size,  # fontname='Arial',
                            color='white')
                    self.field_ax.text(x - 0.95, 53.3 - 5, str(numb - 10),
                            horizontalalignment='center',
                            fontsize=yard_numbers_size,  # fontname='Arial',
                            color='white', rotation=180)

        self.field_ax.set_xlim(self.start_x, self.stop_x)
        self.field_ax.set_ylim(0, self.start_y)
        self.field_ax.set_xticks(range(self.start_x,self.stop_x, 10))

    @staticmethod
    def set_axis_plots(ax, max_x, max_y) -> None:
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

        ax.set_xlim([0, max_x])
        ax.set_ylim([0, max_y])
        
    @staticmethod
    def convert_orientation(x):
        return (-x + 90)%360
    
    @staticmethod
    def polar_to_z(r, theta):
        return r * np.exp( 1j * theta)
    
    @staticmethod
    def deg_to_rad(deg):
        return deg*np.pi/180
        

In [48]:
# from importlib import reload

# def importOrReload(module_name, *names):
#     import sys

#     if module_name in sys.modules:
#         reload(sys.modules[module_name])
#     else:
#         __import__(module_name, fromlist=names)

#     for name in names:
#         globals()[name] = getattr(sys.modules[module_name], name)
        
# importOrReload("temp", "AnimatePlay")
